created a customer_review column with randomly generated sample reviews with no underlying assumption because there was no reviews linked to the transactions to be found anywhere else online. We then perform sentiment analysis to each review and calculate sentiment scores. For each resulting sentiment score, it is a dictionary that contains various scores and we are only interested in the compound score which represents the overall sentiment of a review where it is a single value,  ranging from -1 (most negative) to +1 (most positive). We then apply lambda function to extract the overall qualitative sentiment for each review (positive/neutral/negative).

Afterwards, topic modelling is used to identify common issues and suggestions from customer feedback. The spaCy library is imported for text processing. The en_core_web_sm model is downloaded from spaCy, which provides English language processing capabilities.
The CountVectorizer from sklearn.feature_extraction.text is used to convert text data into a matrix of token counts (bag-of-words model). In essence, it converts text data into a numerical format that LatentDirichletAllocation (LDA) can understand. The LDA from sklearn.decomposition is imported for performing topic modeling. preprocess_text function is defined to tokenize the text, remove stop words as well as punctuations, and lemmatize words. The cleaned text is returned as a single string, ready for topic modelling. This is a critical step before applying Latent Dirichlet Allocation (LDA), which helps in identifying common themes and issues mentioned by customers in their feedback.

The following code topic models the reviews as a whole on the ecommerce platform. Preprocessing is only done for 10000 reviews because the algorithm will over run the time for all 1000000 reviews. CountVectorizer is used to convert text to a document-term matrix. max_df=0.90 means words that appear in more than 90% of the documents will be ignored. min_df=2 means words that appear in fewer than 2 documents will also be ignored. Afterwards we resort to LDA to extract common topics. The n_components parameter determines the dimensionality of the new feature space that LDA projects the data onto. In essence, print_topics is a function defined to display the popular topics based on ALL the reviews (in route 1). There is also code provided (route 2) which displays popular topics for each (store_region, supplier) pair group. This is useful for businesses to see which store in whichever region with their respective supplier needs room for improvement.


In [4]:
# Natural language processing to analyze customer reviews and feedback

import pandas as pd
import random

df = pd.read_csv('final_data.csv', delimiter=",", encoding='ISO-8859-1')

# using sample_reviews to randomly generate data for new column 'customer_review'
sample_reviews = [
    "Great product, very satisfied!",
    "Terrible service, not happy at all.",
    "Fast delivery, packaging was good.",
    "Product is okay, but could be better.",
    "Excellent customer service and quality.",
    "The product arrived late, but it's good.",
    "Not worth the money, poor quality.",
    "Loved it! Would recommend to others.",
    "Decent product for the price.",
    "Delivery was slow, but customer support helped a lot."
]

#ensure reproducibility
random.seed(119)

df['customer_review'] = [random.choice(sample_reviews) for _ in range(len(df))]

print(df.head())

# Setting up sentiment analysis
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(review):
    return analyzer.polarity_scores(review)

df['sentiment_scores'] = df['customer_review'].apply(get_sentiment)

# Extract overall sentiment (positive/negative/neutral)
df['compound'] = df['sentiment_scores'].apply(lambda score: score['compound'])
df['sentiment'] = df['compound'].apply(lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral')

print(df[['customer_review', 'sentiment']])

C:\Users\covan\AppData\Local\Temp\ipykernel_25008\442618712.py:6: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_data.csv', delimiter=",", encoding='ISO-8859-1')


  customer_key  quantity_purchased  total_price purchase_date  \
0      C002181                   4         56.0    2014-01-20   
1      C005518                   4         64.0    2014-01-20   
2      C002449                   6        102.0    2014-01-20   
3      C005284                   4         52.0    2014-01-20   
4      C007996                   6        102.0    2014-01-20   

  time_of_purchase                                     item_name  \
0         14:06:00        Chinet Comfort Hot Cups with Lids 16oz   
1         14:06:00            Chobani Greek Yogurt Variety Pack    
2         14:06:00                  Folgers Classic Roast Coffee   
3         14:06:00                     Spunkmeyer Muffin Variety   
4         14:06:00  Nature Valley Biscuit Sandwich Variety Pack    

           description  unit_price manufacturing_country            supplier  \
0  Dishware - Cups Hot        14.0         United States           MAESA SAS   
1       Food - Healthy        16.0      


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                          customer_review sentiment
0                 Excellent customer service and quality.  positive
1                      Fast delivery, packaging was good.  positive
2                           Decent product for the price.   neutral
3                           Decent product for the price.   neutral
4                      Not worth the money, poor quality.  negative
...                                                   ...       ...
998737            Excellent customer service and quality.  positive
998738  Delivery was slow, but customer support helped...  positive
998739            Excellent customer service and quality.  positive
998740               Loved it! Would recommend to others.  positive
998741                 Fast delivery, packaging was good.  positive

[998742 rows x 2 columns]


In [ ]:
# Topic modelling to identify common issues and suggestions from customer feedback

!pip install spacy gensim
!python -m spacy download en_core_web_sm

import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Load spaCy's English tokenizer
nlp = spacy.load("en_core_web_sm")

# preprocessed text is returned as a single string
def preprocess_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Lemmatization reduces words to their base forms thus making subsequent analysis more consistent.

In [6]:
# Route 1: No groupby, topic models the reviews as a whole on the ecommerce platform

# rationale for choosing only 10000 rows: too many customer_reviews (1000000) to process which causes preprocessing and LDA to overrun time
# Preprocess each review
preprocessed_reviews = [preprocess_text(i) for i in df['customer_review'].head(10000)]

# Vectorize the preprocessed reviews using CountVectorizer to convert text data into a numerical format that LDA can understand
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english') #max_df=0.90 means words that appear in more than 90% of the documents will be ignored while min_df=2 means words that appear in fewer than 2 documents will also be ignored.
#max_df removes words that are too frequent and is unlikely to carry meaningful information
#min_df removes super rare words which could be typos or noise
X = vectorizer.fit_transform(preprocessed_reviews)

# Latent Dirichlet allocation (LDA) to extract popular topics
lda = LDA(n_components=2, random_state=42)
lda.fit(X)

# This function displays the popular topics
def print_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names_out() # Essentially, this is a list of all the words in the vocabulary that the model uses
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Top 5 words that are most representative of each topic
print_topics(lda, vectorizer, 5)

# set to 2 topics and 5 words each such that one can get a sense of the main themes without overwhelming details, prevents information overload and ensures the topics are interpretable

# It can be seen from topic 0 and 1 that 'delivery' and 'slow' is often associated with each other,
# as well as 'money'+'worth' and 'quality'+'poor'. This suggest that there needs to be improvements for delivery
# time, and ensure better quality of products. Good job on keeping the prices affordable for people to point out
# that it is worth their money!

# All in all, businesses can gain a deeper understanding of their customers’ experiences,
# identify pain points, and respond more effectively to customer needs

Topic 0:
customer delivery service quality excellent
Topic 1:
product satisfied great late arrive


In [ ]:
# Route 2: Group reviews by (store_region, supplier) pair group

# Preprocess each customer_review
# rationale for choosing only 10000 rows: too many customer_reviews (1000000) to process which causes preprocessing and LDA to overrun time
df=df.head(10000)
df['processed_review'] = df['customer_review'].apply(preprocess_text)

grouped_reviews = df.groupby(['store_region', 'supplier'])['processed_review'].apply(lambda x: ' '.join(x)).reset_index()

# Helper function to display the popular topics
def print_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))


# Define a function to apply topic modeling for each 'store_region' and 'supplier'
def topic_modeling_for_group(grouped_reviews):
    for _, row in grouped_reviews.iterrows():
        store_region = row['store_region']
        supplier = row['supplier']
        reviews = [row['processed_review']]

        # Vectorize the preprocessed reviews using CountVectorizer
        vectorizer = CountVectorizer(max_df=1, min_df=1, stop_words='english')
        X = vectorizer.fit_transform(reviews)

        # Latent Dirichlet allocation (LDA) to extract popular topics
        lda = LDA(n_components=2, random_state=38)
        lda.fit(X)

        print(f"Store Region: {store_region} | Supplier: {supplier}")

        # Top 5 words that are most representative of each topic for each 'store_region' and 'supplier' combi
        print_topics(lda, vectorizer, 5)


topic_modeling_for_group(grouped_reviews)



# All in all, businesses can gain a deeper understanding of their customers’ experiences,
# identify pain points, and respond more effectively to customer needs. This is useful for businesses
# to see which store in whichever region with their respective supplier needs room for improvement.
